#### Sentiment Analysis Pipeline Code



In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize the Groq API with your key
groq_api_key = "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"
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="gemma-7b-it"
)

# Function to perform sentiment analysis
def analyze_sentiment(text_review, language):
    # Define the improved prompt template
    template = """
    You are a machine learning model trained to classify the sentiment of a review in the '{option}' language. 
    Your task is to classify the sentiment into one of the following categories:
    - "positive" if the sentiment is positive.
    - "negative" if the sentiment is negative.
    - "null" if the sentiment is unclear or not valuable.

    Provide the sentiment as one of the following values only: positive, negative, null.

    Review text: '''{text_review}'''
    """
    
    # Prepare the prompt
    prompt = PromptTemplate(template=template, input_variables=["text_review", "option"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    # Run the prompt through the chain
    response = llm_chain.run({"text_review": text_review, "option": language})

    # Debug: Print the response from the model
    print(f"Model Response: {response}")
    
    # Handling the response to ensure it matches the expected sentiment values
    sentiment = response.strip().lower()  # remove whitespace and convert to lowercase
    if sentiment not in ["positive", "negative", "null"]:
        print(f"Unexpected sentiment value: {sentiment}")
        sentiment = "null"  # Default to "null" if the value is unexpected
    
    return sentiment

# Evaluate the results
def evaluate_results(y_true, y_pred):
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision (macro)": precision_score(y_true, y_pred, average='macro', zero_division=0),
        "Recall (macro)": recall_score(y_true, y_pred, average='macro', zero_division=0),
        "F1 Score (macro)": f1_score(y_true, y_pred, average='macro', zero_division=0),
        "Precision (weighted)": precision_score(y_true, y_pred, average='weighted', zero_division=0),
        "Recall (weighted)": recall_score(y_true, y_pred, average='weighted', zero_division=0),
        "F1 Score (weighted)": f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }
    return metrics

# Example dataset with more variance (positive, negative, neutral reviews)
dataset = [
    {"review": "I loved the product, it's amazing!", "language": "English", "label": "positive"},
    {"review": "Terrible experience, very dissatisfied.", "language": "English", "label": "negative"},
    {"review": "El producto es excelente y cumple lo prometido.", "language": "Spanish", "label": "positive"},
    {"review": "El servicio fue horrible.", "language": "Spanish", "label": "negative"},
    {"review": "The product was okay, but not great.", "language": "English", "label": "neutral"},
    {"review": "I'm really happy with the service, exceeded my expectations.", "language": "English", "label": "positive"},
    {"review": "Meh, it was just alright. Nothing special.", "language": "English", "label": "neutral"},
    {"review": "Horrible! I will never buy from here again!", "language": "English", "label": "negative"},
    {"review": "Un servicio excelente, totalmente recomendable.", "language": "Spanish", "label": "positive"},
    {"review": "El producto no cumplió con lo prometido, muy decepcionado.", "language": "Spanish", "label": "negative"},
    {"review": "The movie was kind of boring, not what I expected.", "language": "English", "label": "neutral"},
    {"review": "Absolutely fantastic! Will recommend it to everyone.", "language": "English", "label": "positive"},
    {"review": "Bad customer support, took too long to respond.", "language": "English", "label": "negative"},
    {"review": "Buen producto, pero le falta algo.", "language": "Spanish", "label": "neutral"},
    {"review": "The experience was just okay, not terrible but not great either.", "language": "English", "label": "neutral"},
    {"review": "Excelente producto, me encantó mucho.", "language": "Spanish", "label": "positive"},
    {"review": "Not worth the price. Really disappointing.", "language": "English", "label": "negative"},
]

# Evaluate metrics (same as before)
y_true = [item["label"] for item in dataset]
y_pred = []

for data in dataset:
    sentiment = analyze_sentiment(data["review"], data["language"])
    y_pred.append(sentiment)

# Ensure lengths match
assert len(y_true) == len(y_pred), "Mismatched lengths between y_true and y_pred"

# Evaluate metrics
metrics = evaluate_results(y_true, y_pred)

# Display results
print("\nEvaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")


Model Response: positive
Model Response: negative
Model Response: positive
Model Response: negative
Model Response: negative
Model Response: positive
Model Response: null
Model Response: negative
Model Response: positive
Model Response: negative
Model Response: negative
Model Response: positive
Model Response: negative
Model Response: positive
Model Response: null
Model Response: negative
Model Response: negative

Evaluation Metrics:
Accuracy: 0.65
Precision (macro): 0.38
Recall (macro): 0.46
F1 Score (macro): 0.41
Precision (weighted): 0.53
Recall (weighted): 0.65
F1 Score (weighted): 0.58
